# ESE-2000 Lab 6
TO DO add intro


## Word Embeddings

A simple approach is to encode words in the index of a long vector. Formally, suppose that we are given a collection of texts that collectively contain a total of $c$ words. We then consider a set of $c$ vectors $\mathbf{e}_i$ whose length is also $c$. These vectors have all zero entries except for the $i$th entry which we set to 1:

$$
(\mathbf{e}_i)_i = 1 \quad \text{~and~} \quad 
(\mathbf{e}_i)_j = 0 \text{~~for~~} i \neq j.
$$

We use the vector $\mathbf{e}_i$ to encode the $i$th word in the corpus.

In corpuses used in practice, we have thousands of different words and anywhere between hundreds of thousands to trillions of sentences. In this lab, we work with a subset of Shakespeare's plays which contains $c = 14,295$ different words and a total of 292,072 words in the corpus. But to illustrate ideas, let us work with a corpus made up of just two quotes:

> *If by your art, my dearest father, you have put the wild waters in this roar, allay them.*  
> *Sir, are not you my father?*

In this corpus, we have a total of 24 different words including 3 punctuation marks. We therefore represent the words in the corpus with $c = 24$ vectors of length $c = 24$. Proceeding in the order of the sentence, the vector $\mathbf{e}_1 = [1, 0, \ldots, 0]$ represents the word “If,” the vector $\mathbf{e}_2 = [0, 1, 0, \ldots, 0]$ represents the word “by,” and so on. The word “father” is the eighth word that appears in the sentence and is therefore represented by the vector $\mathbf{e}_8$. This vector's value at index 8 is $(\mathbf{e}_8)_8 = 1$ and all of its other entries are zero.

When the same word appears again in the corpus, we encode it with the same vector. For example, when the word “father” appears a second time, we still encode it with the vector $\mathbf{e}_8$. This also happens with the comma (“,”) which appears three times and is encoded with the vector $\mathbf{e}_5$ in all three appearances, and with the words “my” and “you” that appear twice and are encoded in the vectors $\mathbf{e}_6$ and $\mathbf{e}_9$. So encoded, our corpus becomes:

> $\mathbf{e}_1 \quad \mathbf{e}_2 \quad \mathbf{e}_3 \quad \mathbf{e}_4 \quad \mathbf{e}_5 \quad \mathbf{e}_6 \quad \mathbf{e}_7 \quad \mathbf{e}_8 \quad \mathbf{e}_5 \quad \mathbf{e}_9 \quad \mathbf{e}_{10}$  
> $\mathbf{e}_{11} \quad \mathbf{e}_{12} \quad \mathbf{e}_{13} \quad \mathbf{e}_{14} \quad \mathbf{e}_{15} \quad \mathbf{e}_{16} \quad \mathbf{e}_{17} \quad \mathbf{e}_5$  
> $\mathbf{e}_{18} \quad \mathbf{e}_{19} \quad \mathbf{e}_{20}$  
> $\mathbf{e}_{21} \quad \mathbf{e}_5 \quad \mathbf{e}_{22} \quad \mathbf{e}_{23} \quad \mathbf{e}_9 \quad \mathbf{e}_6 \quad \mathbf{e}_8 \quad \mathbf{e}_{24}$

This is a defilement of Shakespeare's work. However, this representation of the corpus can be processed with numerical techniques.

Encoding language with these index vectors is not creative and does not work well. We discuss more interesting and useful word embeddings in the next section.


**Task 6.1** 

Get the data for this lab from [dsd.seas.upenn.edu/lab6](https://dsd.seas.upenn.edu/lab6) and load it into your environment. This is a text file containing around 40,000 lines of dialogue from Shakespeare's plays. Split the text into words, defined here to include punctuation marks and line breaks. We associate words with vectors $\mathbf{e}_i$ as in equation $(1)$. Since it would be wasteful to store vectors in which all but one entry is 0, we just store the index of the vector that represents each individual word. For example, if “father” is represented by the index vector $\mathbf{e}_8$, we do not store $\mathbf{e}_8$ to represent this word; we just store the index $i=8$.

Implement a function that turns a word into an index and the inverse function that turns an index into a word. We recommend that you use the code that we provide for this task. It is a somewhat cumbersome and not very enlightening activity.


## Cooccurrence Matrices

To create richer word embeddings, we leverage the cooccurrence matrix $\mathbf{C}$. To construct this matrix, we consider a window of length $W + 1$ and scan the corpus for joint occurrences of words $\mathbf{e}_i$ and $\mathbf{e}_j$. The cooccurrence $C_{ij}$ is the number of times that $\mathbf{e}_j$ appears in a window centered at $\mathbf{e}_i$. If we index the corpus by an index $t$ and use $\mathbf{w}_t$ to represent the $t$th word in the corpus, we can write cooccurrences as:

$$
C_{ij} = \sum_t \mathbb{I}(\mathbf{w}_t = \mathbf{e}_i) = \sum_{u = -W/2}^{u = W/2} \mathbb{I}(\mathbf{w}_u = \mathbf{e}_j),
$$

where we assume that the window is even for simplicity. In the above equation, the first indicator function $\mathbb{I}(\mathbf{w}_t = \mathbf{e}_i) = 1$ only when the window is centered at $\mathbf{w}_t$ and $\mathbf{w}_t = \mathbf{e}_i$. The second indicator function $\mathbb{I}(\mathbf{w}_u = \mathbf{e}_j) = 1$ whenever the word $\mathbf{e}_j$ appears in the window centered at $\mathbf{w}_t$. Thus, the second sum counts the number of times that $\mathbf{e}_j$ appears centered in a window centered at $\mathbf{w}_t = \mathbf{e}_i$. The first sum is counting the number of times that $\mathbf{e}_i$ appears in the corpus.

The cooccurrence matrix $\mathbf{C}$ is relevant because related words tend to appear near each other, and they also tend to appear next to words that indicate their relationships. In an extensive corpus, we expect to find several cooccurrences of the words “birds” and “fly,” indicating that these two words are related. We do not expect to see many cooccurrences of “dogs” and “fly” because dogs do not fly. We also expect to see cooccurrences of the words “bird” and “albatross” and of the words “bird” and “swallow,” indicating that there is some relationship between “albatross” and “swallow.”

We highlight that the cooccurrence matrix $\mathbf{C}$ is symmetric:

$$
\mathbf{C} = \mathbf{C}^T 
\quad \Leftrightarrow \quad
C_{ij} = C_{ji}
$$

This is because whenever the word $\mathbf{e}_j$ appears in a window centered at an occurrence of the word $\mathbf{e}_i$, these two words are less than $W/2$ words apart. This implies that the word $\mathbf{e}_i$ must appear in a window centered at an occurrence of the word $\mathbf{e}_j$.


**Task 6.2** 

Compute the cooccurrence matrix for the Shakespeare corpus loaded in Task 6.1. Use a window of length $W=10$.


## Eigenvector Embeddings

A vector $\mathbf{v}_k$ is said to be an eigenvector of the cooccurrence matrix $\mathbf{C}$ if there exists a constant $\lambda_k$ such that 

$$
\mathbf{C} \mathbf{v}_k = \lambda_k \mathbf{v}_k.
$$

Eigenvectors are peculiar vectors because the matrix multiplication $\mathbf{C} \mathbf{e}$ yields a vector that is, in general, quite different from $\mathbf{e}$. In the case of an eigenvector, the product $\mathbf{C} \mathbf{v}_k = \lambda_k \mathbf{v}_k$ is a simple scaling of $\mathbf{v}_k$. All of the components of $\mathbf{v}_k$ are multiplied by the same number.

It is known that the symmetric matrix $\mathbf{C} \in \mathbb{R}^{c \times c}$ has $c$ distinct eigenvectors. It is customary to order the corresponding $c$ eigenvalues from largest to smallest so that $\lambda_k \geq \lambda_\ell$ when $k < \ell$. Since eigenvector $\mathbf{v}_k$ is associated with eigenvalue $\lambda_k$, the eigenvectors inherit this order. When $k < \ell$, eigenvector $\mathbf{v}_k$ is associated with an eigenvalue that is not smaller than the eigenvalue associated with eigenvector $\mathbf{v}_\ell$ — it is most often larger. We will say that eigenvector $\mathbf{v}_k$ is not smaller than eigenvector $\mathbf{v}_\ell$ or that $\mathbf{v}_k$ is larger than $\mathbf{v}_\ell$ if $\lambda_k > \lambda_\ell$.

It is also customary to group eigenvectors in the eigenvector matrix 

$$
\mathbf{V} = [\mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_{c}],
$$

in which column $k$ registers the value of eigenvector $\mathbf{v}_k$. The eigenvector matrix is an $n \times n$ matrix. It has $c$ columns representing $c$ distinct eigenvectors, which have $c$ rows each.

We consider now a number $n \leq c$ and define the *dimensionality reduction* matrix $\mathbf{V}_n$ grouping the first $n$ eigenvectors of $\mathbf{C}$,

$$
\mathbf{V}_n = [\mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_{n}].
$$

This is a tall matrix because it has $c$ rows but only $n$ columns. These columns coincide with the first $n$ columns of $\mathbf{V}$. Instead of storing all eigenvectors, we are storing only the $n$ largest eigenvectors of $\mathbf{C}$.

We use the dimensionality reduction matrix $\mathbf{V}_n$ to construct representations of vectors $\mathbf{e} \in \mathbb{R}^{c}$ in a space of dimensionality $n$. These representations are

$$
\mathbf{x} = \mathbf{V}_n^T \mathbf{e}.
$$

We say that this is a dimensionality reduction because $\mathbf{x} \in \mathbb{R}^{n}$ is a vector with $n$ components, which is (much) smaller than the number of components $c$ of the vector $\mathbf{e} \in \mathbb{R}^{c}$.

We use dimensionality reduction to compute word embeddings. Given the collection of words $\mathbf{e}_i$, we transform them into the collection of embeddings

$$
\mathbf{x}_i = \mathbf{V}_n^T \mathbf{e}_i.
$$

Representations $\mathbf{x}_i$ are preferable to representations $\mathbf{e}_i$ because they have smaller dimensionality. They also turn out to capture some semantic properties in the sense that vectors $\mathbf{x}_i$ and $\mathbf{x}_j$ that are close represent similar words. This is different from the index embeddings $\mathbf{e}_i$ in which comparisons between different vectors $\mathbf{e}_i$ and $\mathbf{e}_j$ have no meaning.

**Task 6.3** 

Compute the first $n = 256$ eigenvectors of the cooccurrence matrix computed in Task 6.2. Use these eigenvectors to compute the eigenvector embeddings of all of the $c$ words in the corpus loaded in Task 6.1. Store the corpus using these eigenvector embeddings. This is the time series with which we will work in subsequent tasks.

## Principal Component Analysis

The principal component analysis (PCA) transform of a vector $\mathbf{e}$ is its projection in the eigenvector space of $\mathbf{C}$, 

$$
\mathbf{y} = \mathbf{V}^T \mathbf{e}.
$$

This is similar to the dimensionality reduction operation in 

$$
\mathbf{x} = \mathbf{V}_n^T \mathbf{e} 
$$

except that we are using all $c$ eigenvectors instead of the largest $n$.

The PCA representation has the important property that it can be undone by multiplication with the eigenvector matrix. I.e., given the PCA transform $\mathbf{y}$, we can recover the original vector $\mathbf{e}$ as

$$
\mathbf{e} = \mathbf{V} \mathbf{y}.
$$

The combination of the PCA transform and its inverse indicates that $\mathbf{e}$ and $\mathbf{y}$ are equivalent representations of the same information. Given $\mathbf{e}$ we can compute $\mathbf{y}$ and given $\mathbf{y}$ we can compute $\mathbf{e}$.

The same is not true of the dimensionality reduction transformation. When going from $\mathbf{e}$ to $\mathbf{x}$ we lose information precisely because we are reducing dimensionality. In this context, it is interesting to implement the dimensionality recovery operation,

$$
\tilde{\mathbf{e}} = \mathbf{V}_n \mathbf{x} = \mathbf{V}_n \left( \mathbf{V}_n^T \mathbf{e} \right),
$$

and ask how close the recovered vector $\mathbf{e}$ is to the original $\tilde{\mathbf{e}}$. The answer is that for word vectors $\mathbf{e}_i$, the error is small. That is, for most word vectors,

$$
\tilde{\mathbf{e}}_i = \mathbf{V}_n \mathbf{x}_i = \mathbf{V}_n \left( \mathbf{V}_n^T \mathbf{e}_i \right) \approx \mathbf{e}_i.
$$

We have a good theoretical understanding of why this happens. In the context of this lab, it suffices for us to verify that this is true empirically. In Figure \ref{fig_pca_coefficients}, we illustrate the values of the PCA components $y_{ik}$ of some representative word vectors $\mathbf{e}_i$. These PCA components are the entries of the PCA transform vectors $\mathbf{y}_i = \mathbf{V}^T \mathbf{e}_i$ using the eigenvectors of the cooccurrence matrix $\mathbf{C}$ of the Shakespeare corpus that we use in this lab. We see that there is little information in PCA components beyond $k \approx 200$. This substantiates the use of $n = 256$ in Task 6.3.

## Language Transformers
We use here a softmax attention transformer with multiple heads to process language sequences. For reference, a transformer with multiple heads is defined by the recursion,

$$
\mathbf{A}_\ell^h = \text{sm} \left( (\mathbf{Q}^h_\ell \mathbf{X}_{\ell-1})^T (\mathbf{K}^h_\ell \mathbf{X}_{\ell-1}) \right), 
$$

$$
\mathbf{Y}_\ell^h = \mathbf{W}_\ell^h{}^T \mathbf{V}^h_\ell \mathbf{X}_{\ell-1} \mathbf{A}_\ell^h{}^T,
$$

$$
\mathbf{X}_\ell = \mathbf{X}_{\ell-1} + \sigma \left( \sum_{h=1}^H \mathbf{Y}_\ell^h \right).
$$

The input to the transformer is a sequence of $T$ eigenvector word embeddings $\mathbf{X}_0 = \mathbf{X}$ and its output $\mathbf{X}_L = \Phi (\mathbf{X}, \mathcal{A})$ is another sequence of $T$ eigenvector word embeddings. The trainable tensor $\mathcal{A} = \{\mathbf{Q}^h_\ell, \mathbf{K}^h_\ell, \mathbf{V}^h_\ell, \mathbf{W}^h_\ell\}$ contains all of the query, key, value, and dimension recovery matrices of all heads and layers. We use the output sequence to predict the next word, $\mathbf{x}_T$, in the sequence in Section \ref{sec_language_readout}.

Equation 

$$
\mathbf{A}_\ell^h = \text{sm} \left( (\mathbf{Q}^h_\ell \mathbf{X}_{\ell-1})^T (\mathbf{K}^h_\ell \mathbf{X}_{\ell-1}) \right)
$$

is the computation of softmax attention coefficients $\mathbf{A}_\ell^h$ for Layer $\ell$ and Head $h$. We use these attention coefficients to create contextual representations $\mathbf{Y}_\ell^h$ in 

$$
\mathbf{Y}_\ell^h = \mathbf{W}_\ell^h{}^T \mathbf{V}^h_\ell \mathbf{X}_{\ell-1} \mathbf{A}_\ell^h{}^T.
$$

The output of Layer $\ell$ is computed by summing all heads and passing the output through a nonlinear operation in 

$$
\mathbf{X}_\ell = \mathbf{X}_{\ell-1} + \sigma \left( \sum_{h=1}^H \mathbf{Y}_\ell^h \right).
$$

We also add the skip connection $\mathbf{X}_{\ell-1}$ to the output of Layer $\ell$ of the transformer.

Recall that in the attention and representation equations we create the intermediate representations $\mathbf{Q}^h_\ell \mathbf{X}_{\ell-1}$ (queries), $\mathbf{K}^h_\ell \mathbf{X}_{\ell-1}$ (keys), and $\mathbf{V}^h_\ell \mathbf{X}_{\ell-1}$ (values) which are of dimension $m \ll n$. In this lab and in language models in general, the reduction of dimension is aggressive. We have here that $n = 256$ at the input and choose $m = 2$ for intermediate representations.

### Task 6.4

Code a Pytorch module to implement the language transformer as specified by $\text{(13)-(15)}$.
This transformer takes sequences of length $T$ and dimension $n$ as inputs and produces sequences of length $T$ and dimension $n$ as outputs. Make the number of layers $L$ and the number of heads $H$ parameters of the transformer. Queries, keys, and values are of dimension $m$, which is also a parameter of the transformer. Use ReLU nonlinearities at each layer.

This is the same transformer of Lab 5. It is a copy and paste task. That the time series represents language is irrelevant.

### Next Word Prediction

To predict word $\mathbf{x}_T$, we read the output $\mathbf{X}_L = \Phi (\mathbf{X}, \mathcal{A})$ of the transformer. A possible approach is to take the average across time. To set up this readout strategy, let $\mathbf{X}_u$ denote a sequence of $T$ words—in the form of eigenvector embeddings—starting at time $u$,

$$
\mathbf{X}_u = \big[\, \mathbf{x}_u, \mathbf{x}_{u+1}, \ldots, \mathbf{x}_{T + u -1} \,\big] = \mathbf{x}_{u: u + T-1} .
$$

This is a recorded history of the language sequence. Our goal is to predict the next word $\mathbf{x}_{u+T}$ using this recorded history. We do that using the average of the output of the transformer,

$$
\hat{\mathbf{x}}_{u+T} =  \Big[ \, \Phi (\mathbf{X}_u, \mathcal{A}) \, \Big] \mathbf{1}.
$$

We then train the tensor $\mathcal{A} = \{\mathbf{Q}^h_\ell, \mathbf{K}^h_\ell, \mathbf{V}^h_\ell, \mathbf{W}^h_\ell\}$ to maximize prediction accuracy over the corpus. Utilizing a mean squared error (MSE), the prediction task reduces to

$$
\mathcal{A}^* = \arg\min_\mathcal{A ~} \frac{1}{C}~\sum_{u=0}^{C-1} ~ \Big\| \, \Phi \big(\, \mathbf{X}_{u}, \, \mathcal{A} \, \big) \mathbf{1} - \mathbf{x}_{u+T} \,\Big \|^2 \, .
$$

In this equation, we compare the estimate $\hat{\mathbf{x}}_{u+T}$ read out from the transformer's output with the true next word $\mathbf{x}_{u+T}$. We average the resulting MSE loss over the corpus and seek the tensor $\mathcal{A}^*$ that minimizes it. Notice that to simplify notation, we sum over the whole corpus. In reality, we can't predict the last $T$ words because we are using histories $\mathbf{X}_u$ of length $T$. In fact, we have several other limitations in the construction of the training dataset. We may, e.g., want to avoid running over the end of a play or the end of an act. We choose to ignore these practicalities as they have little effect.

### Task 6.5

Split the corpus loaded in Task 6.3 into a training set containing 90% of the total number of words and a test set containing 10% of the words. Recall that this is a time series of word embeddings. Use this training set to train a transformer that predicts the next word embedding using the loss in 

$$
\mathcal{A}^* = \arg\min_\mathcal{A ~} \frac{1}{C}~\sum_{u=0}^{C-1} ~ \Big\| \, \Phi \big(\, \mathbf{X}_{u}, \, \mathcal{A} \, \big) \mathbf{1} - \mathbf{x}_{u+T} \,\Big \|^2 \, .
$$

Use $\( T = 32 \)$ for the length of the history $\( \mathbf{X}_u \)$. Transformer parameters are your choice. If you need a recommendation, use $\( L = 6 \)$, $\( H = 8 \)$, and $\( m = 64 \)$.

Evaluate the test MSE and compare it to the train MSE. Both of these MSE values indicate good prediction. However, this does not mean that we are making good predictions of the next word in the sequence. Explain.

### 6. Probability Readout

The predictions $\(\hat{\mathbf{x}}_{u+T}\)$ in 

$$
\hat{\mathbf{x}}_{u+T} = \Big[ \, \Phi (\mathbf{X}_u, \mathcal{A}) \, \Big] \mathbf{1}
$$ 

may have a small MSE when compared to the observed words $\(\mathbf{x}_{u+T}\)$ but they are not a good strategy for estimating the next word. This is because $\(\hat{\mathbf{x}}_{T}\)$ need not be a valid word. Indeed, it most likely will not be a valid word.

Word $\(\mathbf{e}_i\)$ is represented by the eigenvector encoding $\(\mathbf{x}_i = \mathbf{V}_n^T \mathbf{e}_i\)$ as stated in 

$$
\mathbf{x}_i = \mathbf{V}_n^T \mathbf{e}_i.
$$ 

Since there are a total of $\(c\)$ words in our corpus, there are a total of $\(c\)$ vectors $\(\mathbf{x}_i\)$ that represent valid words. The vectors at the output of the transformer are most unlikely to be one of these vectors, and the estimate $\(\hat{\mathbf{x}}_{T}\)$ is just as unlikely unless we manage to drive the train and test MSEs to zero.

To solve this problem, we must force the readout to be a valid word. We do that with a readout layer whose output is a vector of $\(\tilde{d}_n\)$ probabilities for each of the $\(\tilde{d}_n\)$ words in the corpus. This readout layer is a softmax applied to the output of a fully connected layer that acts on the output of the transformer,

$$
\boldsymbol{\pi} (\mathbf{X}) = \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big( \Phi (\mathbf{X}, \mathcal{A})\big) \, \Big] .
$$ 

The matrix $\(\mathbf{A}\)$ is a trainable parameter with $\(nT\)$ columns and $\(c\)$ rows. After applying the softmax normalization, the entries of the output $\(\boldsymbol{\pi}(\mathbf{X})\)$ add up to one and can be interpreted as a set of probabilities that dictate the likelihood of the next word in the sequence. The $\(i\)$th entry $\(\boldsymbol{\pi}_i(\mathbf{X})\)$ is the predicted probability that the next word is $\(\mathbf{e}_i\)$.

We refer to the probabilities in 

$$
\boldsymbol{\pi} (\mathbf{X}) 
$$ 

as a policy. To train this policy, we minimize the cross-entropy loss between the true word at time $\(u+T\)$ and the probabilities $\(\boldsymbol{\pi}(\mathbf{X})\)$,

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \big(\mathbf{e}_{u+T}\big)^T \big( \log \boldsymbol{\pi}(\mathbf{X}_u) \big) .
$$ 

Notice that in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \big(\mathbf{e}_{u+T}\big)^T \big( \log \boldsymbol{\pi}(\mathbf{X}_u) \big) .
$$ 

the vector $\(\mathbf{e}_{u+T}\)$ is the index encoding of the word at time $\(u+T\)$. This is a vector with all zeros except that it has a 1 at the entry that corresponds to the index of the word that is observed at time $\(u+T\)$. It is therefore a valid probability index that we can incorporate into a cross-entropy comparison.

Further notice that the optimization is joint over the trainable parameters $\(\mathcal{A}\)$ of the transformer and the readout matrix $\(\mathbf{A}\)$. These two parameters are implicit in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \big(\mathbf{e}_{u+T}\big)^T \big( \log \boldsymbol{\pi}(\mathbf{X}_u) \big) .
$$ 

They appear because $\(\boldsymbol{\pi} (\mathbf{X}_u)\)$ depends on $\(\mathbf{A}\)$ and $\(\mathcal{A}\)$. In the hope that it is revealing to make this dependence explicit, we instantiate $\(\mathbf{X} = \mathbf{X}_u\)$ in 

$$
\boldsymbol{\pi} (\mathbf{X}) = \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big( \Phi (\mathbf{X}, \mathcal{A})\big) \, \Big]
$$ 

and substitute the result in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \big(\mathbf{e}_{u+T}\big)^T \big( \log \boldsymbol{\pi}(\mathbf{X}_u) \big) .
$$ 

to write

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \Big[\mathbf{e}_{u+T}\Big]^T 
\bigg[ \log \text{sm} 
\Big[\, \mathbf{A} \, 
\text{vec} \big(\, 
\Phi (\mathbf{X}_u, \mathcal{A}) \, \big) \, \Big]\, \bigg] .
$$ 

We solve this empirical risk minimization (ERM) problem to predict the next word in a sequence of text. This prediction is based on observing a history of length $\(T\)$ that is processed by a transformer

$$
\mathbf{X}_\ell = \mathbf{X}_{\ell-1} + \sigma\bigg( \sum_{h=1}^H \mathbf{Y}_\ell^h  \,\bigg) .
$$ 

with a probability readout layer 

$$
\boldsymbol{\pi} (\mathbf{X}) = \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big( \Phi (\mathbf{X}, \mathcal{A})\big) \, \Big] .
$$ 

Different from the readout strategy in 

$$
\hat{\mathbf{x}}_{u+T} = \Big[ \, \Phi (\mathbf{X}_u, \mathcal{A}) \, \Big] \mathbf{1} 
$$ 

and the training procedure in 

$$
\mathcal{A}^* = \arg\min_{\mathcal{A}} \frac{1}{C}~\sum_{u=0}^{C-1} ~ \Big\| \, \Phi \big(\, \mathbf{X}_{u}, \, \mathcal{A} \, \big) \mathbf{1} - \mathbf{x}_{u+T} \,\Big \|^2 \, .
$$ 

the ERM problem in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A},\, \mathbf{A}} ~ \frac{1}{C}~\sum_{u=0}^{C-1} ~ \big(\mathbf{e}_{u+T}\big)^T \big( \log \boldsymbol{\pi}(\mathbf{X}_u) \big) .
$$ 

produces parameters $\(\mathcal{A}^*\)$ and $\(\mathbf{A}^*\)$ that map directly to predictions of actual words.

### Task 6.6

Modify the transformer of Task 6.4 to add the readout layer in 

$$
\boldsymbol{\pi} (\mathbf{X}) = \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big( \Phi (\mathbf{X}, \mathcal{A})\big) \, \Big].
$$ 



### Task 6.7

Split the corpuses loaded in Task 6.1 and Task 6.3 into a training set containing 90% of the total number of words and a test set containing 10% of the words. Recall that these two are equivalent time series except that the information is encoded differently. In Task 6.1, we store words using index encodings, and in Task 6.3, we store words using eigenvector embeddings. We are loading both here because the eigenvector encodings are the input to the transformer, and the index encodings are needed for the crossentropy comparison in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A}, \, \mathbf{A}} \frac{1}{C} \sum_{u=0}^{C-1} \Big[\mathbf{e}_{u+T}\Big]^T \bigg[ \log \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big(\, \Phi (\mathbf{X}_u, \mathcal{A}) \, \big) \, \Big]\, \bigg].
$$ 

Make sure that time indexes match in your data.

Use the training set to train a transformer that predicts next word probabilities using the transformer with readout of Task 6.6. Use $\(T=32\)$ for the length of the history $\(\mathbf{X}_u\)$. Transformer parameters are your choice. If you need a recommendation, use $\(L=??\)$, $\(H=??\)$, and $\(m=??\)$.

Evaluate the crossentropy loss in the test set and compare it to the crossentropy loss in the training set.

### Model Sampling

After solving the empirical risk minimization (ERM) problem in 

$$
\mathcal{A}^*, \mathbf{A}^* = \arg\min_{\mathcal{A}, \, \mathbf{A}} \frac{1}{C} \sum_{u=0}^{C-1} \Big[\mathbf{e}_{u+T}\Big]^T \bigg[ \log \text{sm} \Big[\, \mathbf{A} \, \text{vec} \big(\, \Phi (\mathbf{X}_u, \mathcal{A}) \, \big) \, \Big]\, \bigg],
$$ 

we have trained values $\(\mathcal{A}^*\)$ for the transformer and $\(\mathbf{A}^*\)$ for the probability readout layer. With these trained values, we can execute 

$$
\mathbf{\pi}^* (\mathbf{X}) = \text{sm} \Big[\, \mathbf{A}^* \, \text{vec} \big( \Phi (\mathbf{X}, \mathcal{A}^*)\big) \, \Big]
$$ 

for any given text sequence $\(\mathbf{X}\)$ of length $\(T\)$. The result is the (optimal) vector of probabilities.

This is not yet a word; it is a vector of probabilities that assigns probabilities to each of the $\(c\)$ words in the corpus. To generate a word, we need to implement a *sampling* strategy.

Let us denote as $\(\pi^* (\mathbf{e}_i | \mathbf{X})\)$ the probability of choosing word $\(\mathbf{e}_i\)$. This is the $\(i\)$th entry of the vector of probabilities $\(\mathbf{\pi}\)$. A possible sampling strategy is to sample the word $\(\mathbf{e}_i\)$ with the highest probability:

$$
\hat{\mathbf{e}} = \arg\max_{\mathbf{e}_i} \pi^* (\mathbf{e}_i | \mathbf{X}).
$$ 

Alternatively, we can sample predictions randomly by choosing different words according to their corresponding probabilities. We write

$$
\hat{\mathbf{e}} = \mathbf{e}_i \sim \pi^* (\mathbf{e}_i | \mathbf{X})
$$ 

to signify that we choose $\(\hat{\mathbf{e}} = \mathbf{e}_i\)$ with probability $\(\pi^* (\mathbf{e}_i | \mathbf{X})\)$.

Sampling according to the largest probability (as in the first equation) is a good strategy if we want to predict the next word in the sequence. However, sampling randomly according to word probabilities (as in the second equation) is a better strategy for generating text. Random sampling better imitates the natural variability of human language, and we will use random sampling.

### Task 6.8

Given trained parameters $\(\mathcal{A}^*\)$ and $\(\mathbf{A}^*\)$, implement the following:

(a) A transformer with parameters \$(\mathcal{A}^*\)$ that takes language sequences $\(\mathbf{X}\)$ of length $\(T\)$ as inputs.

(b) A readout layer that postprocesses the output of the transformer to yield a vector of probabilities $\(\mathbf{\pi}^* (\mathbf{X})\)$.

(c) A sampler that takes probabilities $\(\mathbf{\pi}^*(\mathbf{X})\)$ as inputs and returns words $\(\hat{\mathbf{e}}\)$ sampled according to 

$$
\hat{\mathbf{e}} = \mathbf{e}_i \sim \pi^* (\mathbf{e}_i | \mathbf{X}).
$$ 

The transformer and readout implementations are just instances of the transformer and readout modules from Tasks 6.4 and 6.6. The only new piece here is the sampler.

Try your sampler for a few input sequences.

Here's the text converted to Markdown format:

## Language Generation

In the Language Generation section we adopted a transformer to predict the next word of a sequence of length $\( T \)$. We adapt this model to language generation with a rolling execution.

Begin with a language sequence entered by a user, which we call a prompt. From the prompt we construct a time series $\( \mathbf{X}_0 \)$ with the eigenvector encodings of its words

$$
\mathbf{X}_0 = [\mathbf{x}_0, \ldots, \mathbf{x}_{T-1}].
$$

We assume, for simplicity, that this prompt has length $\( T \)$. Using this prompt we predict the next word in the sequence using the policy $\( \boldsymbol{\pi}^* \)$,

$$
\mathbf{x}_T \sim \boldsymbol{\pi}^*(\mathbf{X}_0).
$$

Although the input $\( \mathbf{x}_T \)$ has been *generated* by the policy $\( \boldsymbol{\pi}^* \)$, we reinterpret it as a *given* word. We then roll the prompt backward and append the generated word $\( \mathbf{x}_T \)$ to construct the series

$$
\mathbf{X}_1 = [\mathbf{x}_1, \ldots, \mathbf{x}_{T-1}, \mathbf{x}_{T}].
$$

In this sequence the first $\( T-1 \)$ entries are part of the user prompt. The last one, $\( \mathbf{x}_{T} \)$, has been generated. We ignore this distinction and proceed to estimate word $\( T+1 \)$ as 

$$
\mathbf{x}_{T+1} \sim \boldsymbol{\pi}^*(\mathbf{X}_1).
$$

We then proceed to append this generated word to the time series in the previous equation and roll the series backward. This procedure yields the time series,

$$
\mathbf{X}_2 = [\mathbf{x}_2, \ldots, \mathbf{x}_{T-1}, \mathbf{x}_{T}, \mathbf{x}_{T+1}].
$$

In this time series we have the last $\( T-2 \)$ words of the user prompt and two words that have been generated by policy $\( \boldsymbol{\pi}^* \)$. These are the words $\( \mathbf{x}_{T} \)$ and $\( \mathbf{x}_{T+1} \)$ generated in the previous equations. We again ignore this distinction and generate the next word as,

$$
\mathbf{x}_{T+2} \sim \boldsymbol{\pi}^*(\mathbf{X}_2).
$$

We append word $\( \mathbf{x}_{T+2} \)$ to the time series, roll the time series backward, and use the updated series to predict the next word in the sequence. In general, at generative step $\( u \)$ we take as an input the time series

$$
\mathbf{X}_u = [\mathbf{x}_u, \ldots, \mathbf{x}_{T-1+u}],
$$

in which the last $\( u \)$ samples have been generated—it can be that all of the samples are generated if $\( u \geq T \)$. From this time series we generate the word in position $\( T+u \)$ as,

$$
\mathbf{x}_{T+u} \sim \boldsymbol{\pi}^*(\mathbf{X}_u).
$$

The output of the generative language model is the string of text $\([\mathbf{x}_T, \ldots, \mathbf{x}_{T_{\max}}]\)$ where $\( T_{\max} \)$ is a prespecified limit for the length of the language sequence to be generated. Of course, rather than returning the eigenvector embeddings $\([\mathbf{x}_T, \ldots, \mathbf{x}_{T_{\max}}]\)$ we return the sequence of corresponding words.

## Task 6.9

Implement the generative language model as specified by the recursion 

$$
\mathbf{X}_u = [\mathbf{x}_u, \ldots, \mathbf{x}_{T-1+u}]
$$ 

to 

$$
\mathbf{x}_{T+u} \sim \boldsymbol{\pi}^*(\mathbf{X}_u).
$$ 

Take prompts of length $\( T=64 \)$ as inputs and generate language sequences of length $\( T_{\max}=500 \)$. To make this task more interesting, modify your implementation to accept prompts of any length $\( T' \leq T \)$. This is not difficult because absent words in the prompt can be set to zero.

Try your generative model for some prompts.

## Positional Encoding

The output of a transformer is equivariant to a permutation of the entries of the time series. If we exchange the positions of $\(\mathbf{x}_t\)$ and $\(\mathbf{x}_u\)$, the output of the transformer remains the same, except that $\([\Phi(\mathbf{X}_u, \mathcal{A})]_u\)$ and $\([\Phi(\mathbf{X}_t, \mathcal{A})]_u\)$ also exchange places. Positional encoding is a strategy to break this symmetry so that words can have different effects depending on their positions.

We use oscillations to define positional encodings. For a time series made up of vectors $\(\mathbf{x}_t\)$ with $\(n\)$ entries, we define $\(n/2\)$ frequencies $\(\alpha_i\)$. For each of these frequencies, we define a time series $\(\mathbf{P}\)$ in which the values for time $\(t\)$ and index $\(i\)$ are given by

$$
p_{ti} = 
\begin{cases}
\cos \left(2\pi\, \alpha_{(i+1)/2} \, \frac{t}{T}\right), & \text{if } i \text{ is odd} \\
\sin \left(2\pi\, \alpha_{i/2} \, \frac{t}{T}\right), & \text{if } i \text{ is even}.
\end{cases}
$$

As per the equation above, positional encoding includes sines and cosines of different frequencies in different rows of the positional encoding time series. Odd rows of $\(\mathbf{P}\)$ are cosines of frequency $\(\alpha_{(i+1)/2}\)$. Even rows of $\(\mathbf{P}\)$ are sines of frequency $\(\alpha_{i/2}\)$.

The use of sines and cosines in this context is motivated by the Fourier basis, which has intimate connections with convolution. This is a story for another day.

**[Javier, please finish description. How do we add?]**

### Task 6.10: Modify Transformer with Positional Encoding

Modify the transformer of Task 6.6 to incorporate positional encoding. Use frequencies $\(\alpha_i = ??\)$ for $\(i=1,2,\ldots, n/2\)$.

### Practical Considerations



### Layer Normalization

In order to improve training stability and convergence, one common implementation trick is to normalize the output vectors of a layer to have zero mean and unit variance. This is commonly referred to as *layer normalization*. If $\mathbf{X} = \mathbf{W}^\ell \mathbf{X}_\ell$ is the output of layer $\ell$, the normalized output $\hat{\mathbf{X}}$ at layer $\ell$ is computed as:

$$
\hat{\mathbf{X}}_{ij} = \gamma_i \cdot \frac{\mathbf{X}_{ij} - \mu_i}{\sqrt{\sigma_{i}^2 + \epsilon}} + \beta_i
$$

Here, $\epsilon > 0$ is a small number to avoid dividing by zero, and $\mu$ and $\sigma^2$ are the row-wise mean and variance of the elements $\mathbf{X}_{ij}$ at layer $\ell$:

$$
\mu_i = \frac{1}{n} \sum_{j=1}^n \mathbf{X}_{ij}
$$

$$
\sigma_i^2 = \frac{1}{n} \sum_{j=1}^n (\mathbf{X}_{ij} - \mu_i)^2
$$

The learnable parameters $\gamma_i$ and $\beta_i$ play the role of recovering the mean and the variance. This might seem like we didn't do anything, but now the learnable parameters do not depend on the computation of $\mathbf{X}$. This results in more stable training.

By normalizing each hidden vector, layer normalization helps to mitigate internal covariate shift and ensures more stable gradients during training.

### Task 6.11

**Modify the transformer of Task 6.10 to incorporate layer normalization at the output of each transformer layer. Use the PyTorch function `nn.LayerNorm`.**

### Future Masking

Notice that in Equation $\text{(13)}$, we have attention coefficients for each pair of words in a sequence. This means that our model can potentially learn to have $\mathbf{A}_{ij}$ will have nonzero attention even if the word $\mathbf{w}_j$ is ahead of the word $\mathbf{w}_i$. For tasks such as word generation, this is undesirable: we want to ensure that our attention coefficients only focus on past words, so that we can effectively predict the next one better.

We can apply *future masking* to ensure this. The idea is to reweight $\mathbf{a}_t$ so that the attention weight is zero for all the words beyond $t$:

$$
\mathbf{a}_t = [a_{1t}\ a_{2t}\ \dots a_{tt}\ 0\ \dots\ 0],
$$

while making sure that the nonzero attention coefficients sum to 1.

An implementation trick to achieve this is to manually set the coefficients to $-\infty$ before passing them to softmax:


$$
\mathbf{B}_{ij} = [(\mathbf{Q}\mathbf{X})^T(\mathbf{K}\mathbf{X})]_{ij}
$$

$$
\tilde{\mathbf{B}}_{ij} = 
\begin{cases}
\mathbf{B}_{ij} & \text{if } j > i \\
-\infty & \text{otherwise}
\end{cases}
$$

So for each head in $\text{(13)}$, $\mathbf{A}^h_\ell$ is replaced by 

$$
\tilde{\mathbf{A}}^h_\ell = \text{sm}(\tilde{\mathbf{B}}^h_\ell).
$$

## Task 6.12

**Modify the transformer of Task 6.11 to incorporate future masking at all layers.**


## Training and Generation


### Task 6.13

**Modify the transformer in Task 6.12 to incorporate future masking at all layers.**

### Task 6.14

**Repeat the generative exercise in Task 6.9 using the transformer trained in Task 6.13.**
